In [17]:
from PIL import Image
import io
import numpy as np
from numpy.dtypes import StrDType
from matplotlib import pyplot as plt
from package.utils.logger import logger

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import multivariate_normal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, adjusted_rand_score

import pandas as pd
from IPython.display import display, HTML

In [5]:

# Bootstrap
raw_dataset = np.load('.ds.tiny/dataset.zip')

dataset = {
    'train': {
        'data': [],
        'names': [],
        'labels': [],
        'unique_labels': [],
    },
    'valid': {
        'data': [],
        'names': [],
        'labels': [],
        'unique_labels': [],
    }
}

images_resize_shape = (128,128)

# For each image we have the path from which we extract the name and the label of the image
for dsKey in raw_dataset.keys():
    splittedKey = dsKey.split('/')

    img_type = splittedKey[2]
    img_label = splittedKey[3]
    img_name = splittedKey[4]
    
    img = Image.open(io.BytesIO(raw_dataset[dsKey])).resize(images_resize_shape, Image.Resampling.LANCZOS)
    img_array = np.asarray(img).reshape(images_resize_shape[0]*images_resize_shape[1], 3)
    
    dataset[img_type]['data'].append(img_array)
    dataset[img_type]['names'].append(img_name)
    dataset[img_type]['labels'].append(img_label)

for img_type in dataset.keys():
    dataset[img_type]['data'] = np.asarray(dataset[img_type]['data'])
    dataset[img_type]['names'] = np.asarray(dataset[img_type]['names'])

    dataset[img_type]['unique_labels'], dataset[img_type]['labels'] = np.unique(np.asarray(dataset[img_type]['labels']), return_inverse=True)

    logger.info([f'data shape({img_type})', dataset[img_type]['data'].shape])
    logger.info([f'data labels({img_type})', dataset[img_type]['labels'].shape])
    logger.info([f'data unique labels({img_type})', dataset[img_type]['unique_labels']])


DEFAULT_LOGGER: 2024-06-04 14:27:43,492 | INFO | 2263878544.py:42 ['data shape(train)', (1500, 16384, 3)]
DEFAULT_LOGGER: 2024-06-04 14:27:43,492 | INFO | 2263878544.py:43 ['data labels(train)', (1500,)]
DEFAULT_LOGGER: 2024-06-04 14:27:43,493 | INFO | 2263878544.py:44 ['data unique labels(train)', array(['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel',
       'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu'],
      dtype='<U12')]
DEFAULT_LOGGER: 2024-06-04 14:27:43,499 | INFO | 2263878544.py:42 ['data shape(valid)', (500, 16384, 3)]
DEFAULT_LOGGER: 2024-06-04 14:27:43,500 | INFO | 2263878544.py:43 ['data labels(valid)', (500,)]
DEFAULT_LOGGER: 2024-06-04 14:27:43,500 | INFO | 2263878544.py:44 ['data unique labels(valid)', array(['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel',
       'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu'],
      dtype='<U12')]


In [20]:
# Dimensionality reduction

n_components_to_test = {
    'PCA': [1200],#[3, 10, 50, 100, 200, 500],
    'LDA': [9]#[3, 5, 7, 9]    
}

PCAs_instances = {
    'original': {},
    'grayscale': {}
}

LDAs_instances = {
    'original': {},
    'grayscale': {}
}

PCAs_results = {
    'train': {
        'original': {},
        'grayscale': {}
    },
    'valid': {
        'original': {},
        'grayscale': {}
    },
}

LDAs_results = {
    'train': {
        'original': {},
        'grayscale': {}
    },
    'valid': {
        'original': {},
        'grayscale': {}
    },
}

# Grayscaled data
grayscale_train_images = np.mean(dataset['train']['data'], axis=2)
grayscale_valid_images = np.mean(dataset['valid']['data'], axis=2)

for n_components in n_components_to_test['PCA']:
    PCAs_instances['original'][n_components] = []
    PCAs_instances['grayscale'][n_components] = []

    PCAs_results['train']['original'][n_components] = []
    PCAs_results['train']['grayscale'][n_components] = []
    PCAs_results['valid']['original'][n_components] = []
    PCAs_results['valid']['grayscale'][n_components] = []

    PCA_original = [
        make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components)
        )
    ]
    
    PCA_grayscale = make_pipeline(
        StandardScaler(),
        PCA(n_components=n_components)
    )

    # Multichannel section
    for i in range(3):
        PCA_original[i].fit(dataset['train']['data'][:,:,i]) 
        PCAs_results['train']['original'][n_components].append(PCA_original[i].transform(dataset['train']['data'][:,:,i]))
        PCAs_results['valid']['original'][n_components].append(PCA_original[i].transform(dataset['valid']['data'][:,:,i]))
        logger.info([f'PCA (channel {i}, {n_components} components): explained_variance_ratio sum', np.sum(PCA_original[i][1].explained_variance_ratio_, axis=0)])

    PCAs_instances['original'][n_components].append(PCA_original)

    # Grayscale section
    PCA_grayscale.fit(grayscale_train_images)

    PCAs_results['train']['grayscale'][n_components] = PCA_grayscale.transform(grayscale_train_images)
    PCAs_results['valid']['grayscale'][n_components] = PCA_grayscale.transform(grayscale_valid_images)

    logger.info([f'PCA (grayscale image, {n_components} components): explained_variance_ratio sum', np.sum(PCA_grayscale[1].explained_variance_ratio_, axis=0)])

for n_components in n_components_to_test['LDA']:
    LDAs_instances['original'][n_components] = []
    LDAs_instances['grayscale'][n_components] = []

    LDAs_results['train']['original'][n_components] = []
    LDAs_results['train']['grayscale'][n_components] = []
    LDAs_results['valid']['original'][n_components] = []
    LDAs_results['valid']['grayscale'][n_components] = []

    LDA_original = [
        make_pipeline(
            StandardScaler(),
            LinearDiscriminantAnalysis(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            LinearDiscriminantAnalysis(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            LinearDiscriminantAnalysis(n_components=n_components)
        )
    ]
    
    LDA_grayscale = make_pipeline(
        StandardScaler(),
        LinearDiscriminantAnalysis(n_components=n_components)
    )


    # Multichannel section
    for i in range(3):
        LDA_original[i].fit(dataset['train']['data'][:,:,i], dataset['train']['labels']) 
        LDAs_results['train']['original'][n_components].append(LDA_original[i].transform(dataset['train']['data'][:,:,i]))
        LDAs_results['valid']['original'][n_components].append(LDA_original[i].transform(dataset['valid']['data'][:,:,i]))
        logger.info([f'LDA (channel {i}, {n_components} components): explained_variance_ratio sum', np.sum(LDA_original[i][1].explained_variance_ratio_, axis=0)])

    LDAs_instances['original'][n_components].append(LDA_original)

    # Grayscale section
    LDA_grayscale.fit(grayscale_train_images,dataset['train']['labels'])

    
    LDAs_results['train']['grayscale'][n_components] = LDA_grayscale.transform(grayscale_train_images)
    LDAs_results['valid']['grayscale'][n_components] = LDA_grayscale.transform(grayscale_valid_images)

    logger.info([f'LDA (grayscale image, {n_components} components): explained_variance_ratio sum', np.sum(LDA_grayscale[1].explained_variance_ratio_, axis=0)])


DEFAULT_LOGGER: 2024-06-04 15:13:31,251 | INFO | 2195140652.py:78 ['PCA (channel 0, 1200 components): explained_variance_ratio sum', 0.995446612574062]
DEFAULT_LOGGER: 2024-06-04 15:13:48,118 | INFO | 2195140652.py:78 ['PCA (channel 1, 1200 components): explained_variance_ratio sum', 0.9947216163948556]
DEFAULT_LOGGER: 2024-06-04 15:14:05,381 | INFO | 2195140652.py:78 ['PCA (channel 2, 1200 components): explained_variance_ratio sum', 0.9950718531475843]
DEFAULT_LOGGER: 2024-06-04 15:14:22,367 | INFO | 2195140652.py:88 ['PCA (grayscale image, 1200 components): explained_variance_ratio sum', 0.994628499786816]
DEFAULT_LOGGER: 2024-06-04 15:14:40,558 | INFO | 2195140652.py:125 ['LDA (channel 0, 9 components): explained_variance_ratio sum', 1.0]
DEFAULT_LOGGER: 2024-06-04 15:14:58,417 | INFO | 2195140652.py:125 ['LDA (channel 1, 9 components): explained_variance_ratio sum', 0.9999999999999998]
DEFAULT_LOGGER: 2024-06-04 15:15:15,385 | INFO | 2195140652.py:125 ['LDA (channel 2, 9 components

In [18]:
# Classification - KNN

k_to_test = {
    'PCA': [3, 5, 9, 15, 21, 27, 55, 111, 251],
    'LDA': [3, 5, 9, 15, 21, 27, 55, 111, 251]
}

KNN_PCA_stats = []
KNN_LDA_stats = []

for k_idx, k in enumerate(k_to_test['PCA']):

    KNN_PCA_stats.insert(k_idx,[k])

    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):
        knn = OneVsOneClassifier(KNeighborsClassifier(k))

        knn.fit(PCAs_results['train']['grayscale'][n_components], dataset['train']['labels'])
        preds = knn.predict(PCAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)
        
        # logger.info([f"KNN on PCA (grayscale images, k = {k}, {n_components} components)", "accuracy", accuracy, "precision", precision])

        KNN_PCA_stats[k_idx].insert(n_components_idx + 1,(accuracy, precision))


for k_idx,k in enumerate(k_to_test['LDA']):
    
    KNN_LDA_stats.insert(k_idx,[k])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['LDA']):
        knn = OneVsRestClassifier( KNeighborsClassifier(k))
        
        knn.fit(LDAs_results['train']['grayscale'][n_components], dataset['train']['labels'])
        preds = knn.predict(LDAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        # logger.info([f"KNN on LDA (grayscale images, k = {k}, {n_components} components)", "accuracy", accuracy, "precision", precision])

        KNN_LDA_stats[k_idx].insert(n_components_idx + 1,(accuracy, precision))

KNN_PCA_df = pd.DataFrame(KNN_PCA_stats, columns=['k\\PCA components'] + n_components_to_test['PCA'])
KNN_LDA_df = pd.DataFrame(KNN_LDA_stats, columns=['k\\LDA components'] + n_components_to_test['LDA'])

display(KNN_PCA_df)
display(KNN_LDA_df)


/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/pyth

,k\PCA components,1000,1500
0,3,"(0.136, 0.201)","(0.14, 0.204)"
1,5,"(0.132, 0.171)","(0.128, 0.173)"
2,9,"(0.144, 0.223)","(0.146, 0.21)"
3,15,"(0.144, 0.232)","(0.138, 0.228)"
4,21,"(0.14, 0.245)","(0.142, 0.235)"
5,27,"(0.15, 0.225)","(0.148, 0.22)"
6,55,"(0.162, 0.195)","(0.158, 0.19)"
7,111,"(0.154, 0.267)","(0.156, 0.28)"
8,251,"(0.192, 0.146)","(0.192, 0.158)"


,k\LDA components,3
0,3,"(0.14, 0.16)"
1,5,"(0.134, 0.141)"
2,9,"(0.138, 0.142)"
3,15,"(0.138, 0.138)"
4,21,"(0.144, 0.145)"
5,27,"(0.148, 0.151)"
6,55,"(0.146, 0.144)"
7,111,"(0.142, 0.139)"
8,251,"(0.144, 0.142)"


In [26]:
# SVM

kernels_to_test = {
    'PCA': ['poly'],#['linear', 'poly', 'sigmoid'],
    'LDA': ['poly']#['linear', 'poly', 'sigmoid'],
}

SVM_PCA_stats = []
SVM_LDA_stats = []

for kernel_idx,kernel in enumerate(kernels_to_test['PCA']):

    SVM_PCA_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):#n_components_to_test['PCA']):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(PCAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = svm.predict(PCAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        SVM_PCA_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))


for kernel_idx,kernel in enumerate(kernels_to_test['LDA']):

    SVM_LDA_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['LDA']):
        svm = SVC(kernel=kernel)

        svm.fit(LDAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = svm.predict(LDAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        SVM_LDA_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

SVM_PCA_stats = pd.DataFrame(SVM_PCA_stats, columns=['kernel\\PCA components'] + n_components_to_test['PCA'])
SVM_LDA_stats = pd.DataFrame(SVM_LDA_stats, columns=['kernel\\LDA components'] + n_components_to_test['LDA'])

display(SVM_PCA_stats)
display(SVM_LDA_stats)

,kernel\PCA components,1200
0,poly,"(0.208, 0.275)"


,kernel\LDA components,9
0,poly,"(0.126, 0.135)"


In [22]:
# GMM

GMM_n_components_to_test = {
    'PCA': [15, 30, 100]
}

GMM_PCA_score = []

for GMM_n_components_idx,GMM_n_components in enumerate(GMM_n_components_to_test['PCA']):

    GMM_PCA_score.insert(GMM_n_components_idx,[GMM_n_components])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):
        gmm = GaussianMixture(n_components=GMM_n_components)

        gmm.fit(PCAs_results['train']['grayscale'][n_components])

        preds = gmm.predict(PCAs_results['valid']['grayscale'][n_components])

        rand_score = round(adjusted_rand_score(dataset['valid']['labels'], preds),3)

        GMM_PCA_score[GMM_n_components_idx].insert(n_components_idx + 1,(rand_score))

GMM_PCA_stats = pd.DataFrame(GMM_PCA_score, columns=['components\\PCA components'] + n_components_to_test['PCA'])

display(GMM_PCA_stats)

,components\PCA components,3,10,50,100,200,500
0,15,0.017,0.010,0.010,0.002,-0.000,0.009
1,30,0.013,0.008,0.007,-0.000,0.005,0.010
2,100,0.008,0.010,0.001,0.000,0.019,0.008
